# Stock 工具

主要股票都是用字串的方式處理，格式為 "2330.TW", "GOOG" 台股後面要加上 ".TW"，時間區間則為字串如 `5y` 等

## Import utils Modules

Import 工具模組，安裝與導入

In [ ]:
%pip install pandas yfinance

In [ ]:
import sys
sys.path.append('utils')

import database
import get_info

## 資料庫基本更新

確認與更新資料庫目前資料的版本

In [ ]:
db_path = "../stock_db.db"
db = database.DBManager(db_path)

db.list_stocks()

更新資料庫中的所有資料到最新的狀態

In [ ]:
for stock in db.list_stocks() :
    db.update_stock_in_db(stock)
    print("-----")

## 加入新的股票(如果有需要)

設計上沒有單獨儲存 stock 到資料庫，當試圖從資料庫抓取資料時，沒有拿到對應的資料就會自動去 yfinance 上面撈取並存入資料庫中。

In [ ]:
new_stocks = ["SPY", "MSFT", "INTC", "GOOG", "AMD", "TSLA", "NVDA", "AVGO", "UNH", "QQQ", "AAPL", "AMZN", "U", "CRM", "PLTR", "NFLX", "SPOT"]

for stock in new_stocks:
    db.get_stock(stock, "5y")

In [ ]:
get_info.get_quarter_medians_and_year_avg(db_path, "AMD", 2025)

### 驗證或查看資料庫資料

In [ ]:
import pandas as pd
import sqlite3

# --- 請確認以下設定和你的專案一致 ---
db_path = "../stock_db.db"  # 你的資料庫路徑
stock_ticker = "AMD"       # 你想查詢的股票
year = 2025                # 你想查詢的年份
# ------------------------------------

# 建立查詢第一季資料的 SQL 語法
# 表格名稱預期是 t_AMD (將 '.' 換成 '_')
table_name = f"t_{stock_ticker.replace('.', '_')}"
query = f"""
    SELECT Date, Close FROM {table_name}
    WHERE Date BETWEEN '{year}-01-01' AND '{year}-03-31'
    ORDER BY Date ASC
    LIMIT 5
"""

print(f"正在查詢資料庫：{db_path}")
print(f"執行的 SQL 語法：\n{query}")

try:
    # 連線資料庫並執行查詢
    conn = sqlite3.connect(db_path)
    df_q1 = pd.read_sql(query, conn)
    conn.close()

    # 顯示查詢結果
    print("\n--- 查詢結果 ---")
    if df_q1.empty:
        print(f"資料庫中完全沒有 {stock_ticker} 在 {year} 年第一季的任何資料。")
    else:
        print(f"找到了 {stock_ticker} 在 {year} 年第一季的資料，以下是前幾筆：")
        print(df_q1)

except Exception as e:
    print(f"\n查詢時發生錯誤：{e}")
    print("請確認資料庫路徑是否正確，以及資料表名稱 (t_AMD) 是否存在。")

## 計算年化標準差

### 計算日報酬率

In [ ]:
import pandas as pd
import numpy as np # 我們需要 numpy 來做開根號

# 假設你的資料已經載入到 df
df = db.get_stock("GOOG", "1y") 

# 1. 計算每日報酬率，pandas 有一個超方便的函式 pct_change()
df['Daily_Return'] = df['Close'].pct_change()

# 計算報酬率後，第一天的值會是 NaN (因為沒有前一天的價格)，我們把它去掉
df.dropna(inplace=True)

print("計算出的每日報酬率 (前五筆):")
print(df['Daily_Return'].head())

### 計算每日標準差

In [ ]:
# 2. 計算每日報酬率的標準差
daily_std = df['Daily_Return'].std()

print(f"\n每日標準差: {daily_std:.6f}")

### 計算年化標準差

In [ ]:
# 3. 進行年化 (乘以 252 的平方根)
annualized_std = daily_std * np.sqrt(252)

print(f"年化標準差: {annualized_std:.4f}")
print(f"換算成百分比: {annualized_std:.2%}")

## 年化標準差 - 函數化

In [ ]:
stock = "UNH"

stock_df_5y = db.get_stock(stock, "5y") 
stock_df_3y = db.get_stock(stock, "3y")
stock_df_1y = db.get_stock(stock, "1y") 

In [ ]:
stock_annual_return_1 = get_info.calculate_annual_return(stock_df_1y['Close'])
stock_annual_return_3 = get_info.calculate_annual_return(stock_df_3y['Close'])
stock_annual_return_5 = get_info.calculate_annual_return(stock_df_5y['Close'])
print(f"{stock} 的1年年化報酬率為: {stock_annual_return_1:.2%}")
print(f"{stock} 的3年年化報酬率為: {stock_annual_return_3:.2%}")
print(f"{stock} 的5年年化報酬率為: {stock_annual_return_5:.2%}")

In [ ]:
stock_annual_volatility_1 = get_info.calculate_annualized_std(stock_df_1y['Close'])
stock_annual_volatility_3 = get_info.calculate_annualized_std(stock_df_3y['Close'])
stock_annual_volatility_5 = get_info.calculate_annualized_std(stock_df_5y['Close'])
print(f"{stock} 的1年年化標準差（波動率）為: {stock_annual_volatility_1:.2%}")
print(f"{stock} 的3年年化標準差（波動率）為: {stock_annual_volatility_3:.2%}")
print(f"{stock} 的5年年化標準差（波動率）為: {stock_annual_volatility_5:.2%}")

## 計算 Sharp Ratio

In [ ]:

# 假設當前的無風險利率為 3%
risk_free_rate = 0.045

# 計算 GOOG 的五年夏普值
sharp_ratio_1 = get_info.calculate_sharpe_ratio(stock_df_1y['Close'], risk_free_rate)
sharp_ratio_2 = get_info.calculate_sharpe_ratio(stock_df_3y['Close'], risk_free_rate)
sharp_ratio_3 = get_info.calculate_sharpe_ratio(stock_df_5y['Close'], risk_free_rate)
print(f"{stock} 的1年夏普值為: {sharp_ratio_1:.2f}")
print(f"{stock} 的3年夏普值為: {sharp_ratio_2:.2f}")
print(f"{stock} 的5年夏普值為: {sharp_ratio_3:.2f}")


## 象限決策


### 基準點 : SPY

In [ ]:
# ==============================================================================
# --- 步驟一：動態計算基準 (SPY) 的指標 ---
# ==============================================================================
benchmark_ticker = "SPY"
benchmark_metrics = {}
periods_map = {"5年": "5y", "3年": "3y","1年": "1y"}

print(f"--- 正在計算基準 ({benchmark_ticker}) 指標 ---")
for period_name, period_code in periods_map.items():
    try:
        spy_df = db.get_stock(benchmark_ticker, period_code)
        if not spy_df.empty:
            bench_return = get_info.calculate_annual_return(spy_df['Close'])
            bench_volatility = get_info.calculate_annualized_std(spy_df['Close'])
            benchmark_metrics[period_name] = {"return": bench_return, "volatility": bench_volatility}
        else:
            benchmark_metrics[period_name] = {"return": np.nan, "volatility": np.nan}
    except Exception as e:
        print(f"計算 {benchmark_ticker} {period_name} 指標時發生錯誤: {e}")
        benchmark_metrics[period_name] = {"return": np.nan, "volatility": np.nan}

# 為了方便檢視，將計算出的基準指標也用表格印出
benchmark_df = pd.DataFrame(benchmark_metrics).T
benchmark_df.rename(columns={'return': '年化報酬率', 'volatility': '年化波動率'}, inplace=True)
styled_benchmark_df = benchmark_df.style.format("{:.2%}").set_caption(f"<b>市場基準 ({benchmark_ticker}) 表現</b>")
display(styled_benchmark_df)
print("\n--- 基準指標計算完成 ---")

In [ ]:

# ==============================================================================
# --- 步驟二：分析目標股票，並「帶入」基準數值進行比較 ---
# ==============================================================================
stock = "U"
risk_free_rate = 0.045

print(f"\n--- 正在分析目標股票: {stock} ---")

# 象限分析的輔助函式 (維持不變)
def classify_quadrant(stock_return, stock_vol, bench_return, bench_vol):
    if pd.isna(stock_return) or pd.isna(stock_vol) or pd.isna(bench_return):
        return "資料不足"
    if stock_return >= bench_return:
        return "② 高報酬/低風險 (夢幻逸品)" if stock_vol <= bench_vol else "① 高報酬/高風險 (明星股)"
    else:
        return "③ 低報酬/低風險 (防禦型)" if stock_vol <= bench_vol else "④ 低報酬/高風險 (黑洞區)"

# 計算與分析
metrics = {}
for period_name, period_code in periods_map.items():
    try:
        stock_df = db.get_stock(stock, period_code)
        if not stock_df.empty:
            annual_return = get_info.calculate_annual_return(stock_df['Close'])
            annual_std = get_info.calculate_annualized_std(stock_df['Close'])
            sharpe = get_info.calculate_sharpe_ratio(stock_df['Close'], risk_free_rate)
            
            # ### 關鍵串接點 ###
            # 從預先算好的 benchmark_metrics 字典中，取出對應期間的基準數據
            bench_data = benchmark_metrics[period_name]
            
            # 把「個股指標」和「基準指標」一起傳入函式進行比較
            quadrant_desc = classify_quadrant(annual_return, annual_std, bench_data["return"], bench_data["volatility"])
            
            metrics[period_name] = {
                "年化報酬率": annual_return, "年化波動率": annual_std,
                "夏普值": sharpe, "象限分析": quadrant_desc
            }
        else:
            metrics[period_name] = {"年化報酬率": np.nan, "年化波動率": np.nan, "夏普值": np.nan, "象限分析": "資料不足"}
    except Exception as e:
        print(f"分析 {stock} {period_name} 時發生錯誤: {e}")
        metrics[period_name] = {"年化報酬率": np.nan, "年化波動率": np.nan, "夏普值": np.nan, "象限分析": "資料不足"}

# --- 步驟三：整理並輸出最終表格 ---
results_df = pd.DataFrame(metrics).T
results_df = results_df[["年化報酬率", "年化波動率", "夏普值", "象限分析"]]

styled_df = results_df.style.format({
    "年化報酬率": "{:.2%}", "年化波動率": "{:.2%}", "夏普值": "{:.2f}"
}).set_caption(f"<b>{stock} 的風險與報酬分析 (無風險利率: {risk_free_rate:.1%})</b>")\
  .set_properties(**{'text-align': 'center'})\
  .set_table_styles([
      {'selector': 'caption', 'props': 'caption-side: top; font-size: 1.2em;'},
      {'selector': 'th', 'props': 'text-align: center;'}
  ])\
  .set_properties(subset=['象限分析'], **{'text-align': 'left', 'font-weight': 'bold'})

display(styled_df)